In [35]:
# | default_exp config
# | export
from datetime import datetime
from beforerr.project import produce_or_load_file
from discontinuitypy.datasets import IDsDataset
from space_analysis.meta import PlasmaDataset, Dataset
from space_analysis.utils.speasy import Variables
import polars as pl
from functools import cached_property

from tqdm.auto import tqdm

In [36]:
# | export
def standardize_plasma_data(data: pl.LazyFrame, meta: PlasmaDataset):
    """
    Standardize plasma data columns across different datasets.

    Notes: meta will be updated with the new column names
    """

    if meta.density_col:
        data = data.rename({meta.density_col: "plasma_density"})
        meta.density_col = "plasma_density"
    return data

In [39]:
# | export
def split_timerange(timerange: list[datetime], split: int = 1):
    """
    Split a timerange into multiple timeranges.
    """
    from sunpy.time import TimeRange

    trs: list[TimeRange] = TimeRange(timerange).split(split)
    return [[tr.start.value, tr.end.value] for tr in trs]


class IDsConfig(IDsDataset):
    """
    Extend the IDsDataset class to provide additional functionalities:

    - Split data to handle large datasets
    """

    timerange: list[datetime] = None
    split: int = 1

    @property
    def timeranges(self): 
        return split_timerange(self.timerange, self.split)
    
    @property
    def file_prefix(self):
        if self.timerange is None:
            return super().file_prefix
        else:
            tr_str = "-".join(t.strftime("%Y%m%d") for t in self.timerange)
            return super().file_prefix + f"_tr={tr_str}"

In [ ]:
# | export
def get_vars(self, vars: str, timerange: list[datetime] = None):
    meta: Dataset = getattr(self, f"{vars}_meta")
    timerange = timerange or self.timerange or meta.timerange
    return Variables(
        timerange=timerange,
        provider=self.provider,
        **meta.model_dump(exclude_unset=True),
    )


class SpeasyIDsConfig(IDsConfig):
    """Based on `speasy` Variables to get the data"""

    provider: str = "cda"

    def get_vars(self, *args, **kwargs):
        return get_vars(self, *args, **kwargs)

    def get_vars_df(self, vars: str, **kwargs):
        return get_vars(self, vars, **kwargs).to_polars()

    # Variables
    @cached_property
    def mag_vars(self):
        return self.get_vars("mag")

    @cached_property
    def plasma_vars(self):
        return self.get_vars("plasma")

    def get_data(self):
        # TODO: directly get columns from the data without loading them
        self.plasma_meta.density_col = (
            self.plasma_meta.density_col or self.plasma_vars.data[0].columns[0]
        )
        self.plasma_meta.velocity_cols = (
            self.plasma_meta.velocity_cols or self.plasma_vars.data[1].columns
        )

        self.data = self.data or self.get_vars_df("mag")
        self.plasma_data = self.plasma_data or self.get_vars_df("plasma").pipe(
            standardize_plasma_data, self.plasma_meta
        )
        return self

    def produce_or_load(self, **kwargs):
        if self.split == 1:
            self.file.exists() or self.get_data()
            return super().produce_or_load(**kwargs)
        else:
            updates = [{"timerange": tr, "split": 1} for tr in self.timeranges]
            configs = [self.model_copy(update=update, deep=True) for update in updates]
            datas, files = zip(
                config.produce_or_load(**kwargs) for config in tqdm(configs)
            )

            return produce_or_load_file(
                f=pl.concat,
                c=dict(items=datas),
                file=self.file,
            )